### NOTEBOOK SUMMARY

This notebook contains a summarized way to obtain all models used in the final model of full job characteristic predictions. Additionally, after training and saving each model there are two assembling funtions for the models for full predictions, the first one being notebook prone and the second one a sketch of what a it would look like in a cluster enviromnent. 

The Following code block contains the model used for the process of classification of jobs as deterministic (regular) or nondeterministic (irregular).

In [ ]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestRegressor
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Upload the downloaded dataset 'HPCsyntheticdata.parquet'.
df_classification = pd.read_parquet('/your/path/HPCsyntheticdata.parquet', engine='pyarrow')

# Features used in the training process of the classifier model between regular and irregular jobs.
X_classification = df_classification[['inst', 'bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'qos', 'priority', 'eligible_time_epoch']].values
y_label = df_classification['label'].values  # Assuming 'label' is the target column

# Split data for training and testing (add a validation split optionally as well)
X_train_classification, X_test_classification, y_train_classification, y_test_classification = train_test_split(
    X_classification, y_label, test_size=0.2, random_state=42)

# Scaling data with a MaxAbsScaler for uniformity.
scaler_classification = MaxAbsScaler()
X_train_classification_scaled = scaler_classification.fit_transform(X_train_classification)
X_test_classification_scaled = scaler_classification.transform(X_test_classification)

# Initialize and train ExtraTreesClassifier (defined weights for more false negatives than false positives)
class_weights = {0: 1, 1: 100}
classifier_model = ExtraTreesClassifier(n_estimators=50, class_weight=class_weights, random_state=42, n_jobs=-1)
classifier_model.fit(X_train_classification_scaled, y_train_classification)

# Save the classification model and scaler to be uploaded to the final model.
joblib.dump(classifier_model, 'classifier_model.joblib')
joblib.dump(scaler_classification, 'scaler_classification.joblib')

print("Models saved successfully.")

After the classification process the next step lies on the characterization of the irregular jobs. In the following code block the model predicts the mean power consumption of the job during it's execution.

The dataset is, in order to evaluate only irregular jobs, split by removing all jobs labeled as '1' (regular).

In [ ]:
df_classification = pd.read_parquet('/your/path/HPCsyntheticdata.parquet', engine='pyarrow')
# Filter the dataset to keep only the irregular jobs (labeled as 0).
df_regression_filtered = df_classification[df_classification['label'] == 0].copy()

df_regression_filtered = df_regression_filtered[['inst', 'bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch', 'mean_power','run_time']]

# Save the filtered regression dataset
df_regression_filtered.to_parquet('/your/path/df_completed_filtered_only_0s_meanpower.parquet', engine='pyarrow')

# Load the new dataset.
df_regression = pd.read_parquet('/your/path/df_completed_filtered_only_0s_meanpower.parquet', engine='pyarrow')

# Define features and target for regression
X_regression = df_regression[['inst', 'bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch']].values
y_power = df_regression['mean_power'].values.reshape(-1, 1)  

# Split data for regression (optionally add a validation split)
X_train_regression, X_test_regression, y_train_power, y_test_power = train_test_split(
    X_regression, y_power, test_size=0.2, random_state=42)

# Apply MinMaxScaler for data uniformity
scaler_regression = MinMaxScaler()
X_train_regression_scaled = scaler_regression.fit_transform(X_train_regression)
X_test_regression_scaled = scaler_regression.transform(X_test_regression)

# Scale the target variable (mean_power)
y_scaler = MinMaxScaler()
y_train_power_scaled = y_scaler.fit_transform(y_train_power)
y_test_power_scaled = y_scaler.transform(y_test_power)

# Initialize and train RandomForestRegressor
regression_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
regression_model.fit(X_train_regression_scaled, y_train_power_scaled.ravel())

# Save the regression model and scalers
joblib.dump(regression_model, 'regression_model.joblib')
joblib.dump(scaler_regression, 'scaler_regression.joblib')
joblib.dump(y_scaler, 'y_scaler.joblib')

print("Models saved successfully.")


In addition to predicting Power consumption of the job it is, below, created a model to predict the run time of that same job. In order to do so the jobs are split into the interval of run time they belong to with a multiclassifier model as the range of values in the dataset is very large. 


In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (ExtraTreesClassifier, StackingClassifier, 
                              ExtraTreesRegressor, RandomForestRegressor)
from sklearn.linear_model import LogisticRegression, SGDClassifier, HuberRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from xgboost import XGBClassifier  

# Load the regression dataset with error handling
df = pd.read_parquet('your/path/df_completed_filtered_only_0s_meanpower.parquet', engine='pyarrow')

X = df[['inst', 'bi', 'num_gpus_req', 'mem_req', 
         'num_cores_req', 'num_tasks', 'qos', 
         'priority', 'eligible_time_epoch']].values
y = df['run_time'].values

# Calculation of the key percentiles for multiclass classification
percentile_25th = np.percentile(y, 25)
percentile_50th = np.percentile(y, 50)
percentile_75th = np.percentile(y, 75)
mean_y = np.mean(y)
percentile_95th = np.percentile(y, 95)
percentile_99th = np.percentile(y, 99)  

# Multiclass target based on percentiles
def create_multiclass_target(value):
    if value <= percentile_25th:
        return 0
    elif percentile_25th < value <= percentile_50th:
        return 1
    elif percentile_50th < value <= percentile_75th:
        return 2
    elif percentile_75th < value <= mean_y:
        return 3
    elif mean_y < value <= percentile_95th:
        return 4
    elif percentile_95th < value <= percentile_99th:
        return 5
    else:
        return 6

y_multiclass = np.array([create_multiclass_target(val) for val in y])

# Split the dataset in optional splits 
X_train, X_test, y_train_multiclass, y_test_multiclass = train_test_split(X, y_multiclass, test_size=0.1, random_state=42)

# Scale the features using MinMaxScaler for uniformity
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Base learners used for the stacking model.
base_learners = [
    ('et', ExtraTreesClassifier(n_estimators=500, random_state=42)),
    ('SGD', SGDClassifier(max_iter=3000, tol=1e-4, alpha=1e-5, 
                          early_stopping=True, learning_rate='adaptive', 
                          eta0=0.01, loss='modified_huber', 
                          class_weight='balanced', n_iter_no_change=10)),
    ('xgb', XGBClassifier(n_estimators=500, learning_rate=0.05, 
                          subsample=0.8, random_state=42))
]

# Meta model (Logistic Regression)
meta_model = LogisticRegression(C=1.0, max_iter=3000, solver='lbfgs', multi_class='multinomial')

# Stacking classifier
stacking_clf = StackingClassifier(estimators=base_learners, final_estimator=meta_model)

# Train the stacking classifier
stacking_clf.fit(X_train_scaled, y_train_multiclass)

# Predictions and results if you want to vizualise the accuracy for the test data.
y_test_pred_multiclass = stacking_clf.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test_multiclass, y_test_pred_multiclass)
print(f'Test Accuracy: {test_accuracy:.2f}')

# Save the stacking model and scaler
joblib.dump(stacking_clf, 'multiclass_stacking_classifier_model_6classes.pkl')
joblib.dump(scaler, 'minmax_scaler.pkl')
print("Multiclass model and scaler saved successfully.")


Leveraging on the new smaller intervals each class (interval) is now assigned a scaler and the regressor model that best fits the data in each class in order to finalize the run time prediction.

In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (ExtraTreesClassifier, StackingClassifier, 
                              ExtraTreesRegressor, RandomForestRegressor)
from sklearn.linear_model import LogisticRegression, SGDClassifier, HuberRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from xgboost import XGBClassifier  

df = pd.read_parquet('your/path/df_completed_filtered_only_0s_meanpower.parquet', engine='pyarrow')

X = df[['inst', 'bi', 'num_gpus_req', 'mem_req', 
         'num_cores_req', 'num_tasks', 'qos', 
         'priority', 'eligible_time_epoch']].values
y = df['run_time'].values


# Load saved classifier and scaler
stacking_clf = joblib.load('multiclass_stacking_classifier_model_6classes.pkl')
scaler = joblib.load('minmax_scaler.pkl')

# Predict class labels for regression
X_scaled = scaler.transform(X)
y_pred_classes = stacking_clf.predict(X_scaled)

# Defining the subsets of best performing features per class 

feature_subsets = {
    0: ['num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch'], 
    1: ['inst', 'num_gpus_req', 'num_cores_req', 'qos', 'eligible_time_epoch'], 
    2: ['inst', 'bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'priority', 'eligible_time_epoch'], 
    3: ['inst', 'bi', 'num_gpus_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch'], 
    4: ['inst', 'bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch'],
    5: ['num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch'],
    6: ['bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch']
}


# Initializing dictionaries for scalers and regressors considering the different classes of regressions.
scalers = {}
regressors = {}
train_y_true, train_y_pred, test_y_true, test_y_pred, val_y_true, val_y_pred = [], [], [], [], [], []

# Training regressors per class
for class_label in range(7):
    class_indices = np.where(y_pred_classes == class_label)[0]
    if len(class_indices) == 0:
        print(f"No instances predicted for class {class_label}, skipping.")
        continue
    
    selected_features = feature_subsets.get(class_label, None)
    if not selected_features:
        print(f"No feature subset defined for class {class_label}, skipping.")
        continue

    X_class = df[selected_features].values[class_indices]
    y_class = y[class_indices]

    # Split into train, validation, and test sets 
    X_train, X_temp, y_train, y_temp = train_test_split(X_class, y_class, test_size=0.2, random_state=42)
    X_val, y_val = X_temp[:int(len(X_temp) * 0.5)], y_temp[:int(len(X_temp) * 0.5)]
    X_test, y_test = X_temp[int(len(X_temp) * 0.5):], y_temp[int(len(X_temp) * 0.5):]

    # Definning and fitting the correct scaler for the each class
    if class_label in [0, 1, 2, 3]:
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        X_test_scaled = scaler.transform(X_test)
    elif class_label == 5:
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        X_test_scaled = scaler.transform(X_test)
    else:
        scaler = None 
        X_train_scaled, X_val_scaled, X_test_scaled = X_train, X_val, X_test

    # Save the scalers for future use
    scalers[class_label] = scaler

    # Assigning the appropriate regressor for the each class
    if class_label in [0, 1]:
        regressor = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
    elif class_label in [3, 4]:
        regressor = ExtraTreesRegressor(n_estimators=200, random_state=42, n_jobs=-1)
    elif class_label in [2, 5]:
        regressor = ExtraTreesRegressor(n_estimators=500, random_state=42, n_jobs=-1)
    else:
        regressor = HuberRegressor()

    # Training the regressor for the current class
    regressor.fit(X_train_scaled, y_train)
    regressors[class_label] = regressor  # Save the trained regressor

    # Store the training, validation, and test predictions and actual values if you want to visualize results.
    train_y_true.extend(y_train)
    train_y_pred.extend(regressor.predict(X_train_scaled).astype('int32'))
    test_y_true.extend(y_test)
    test_y_pred.extend(regressor.predict(X_test_scaled).astype('int32'))
    val_y_true.extend(y_val)
    val_y_pred.extend(regressor.predict(X_val_scaled).astype('int32'))

# Calculate the overall mean absolute error if you want to visualize results
overall_mae = mean_absolute_error(np.concatenate((train_y_true, test_y_true, val_y_true)), 
                                   np.concatenate((train_y_pred, test_y_pred, val_y_pred)))
print(f"Overall Mean Absolute Error: {overall_mae:.2f}")

# Save each trained regressor model and the corresponding scaler for each class
for class_label in range(7):
    if class_label not in regressors:
        continue
    joblib.dump(regressors[class_label], f'regressor_model_class_{class_label}.pkl')
    if scalers[class_label]:
        joblib.dump(scalers[class_label], f'scaler_class_{class_label}.pkl')
    print(f'Regressor and scaler (if applicable) for class {class_label} saved successfully.')

Let us now obtain a sample of each type of job in order to run in the fully built model to exemplify its behaviour.

In [11]:
import pandas as pd
import random

# Load the dataset
df1 = pd.read_parquet('HPCsyntheticdata.parquet', engine='pyarrow')

# Filter the dataframe by label
df_label_0 = df1[df1['label'] == 0]
df_label_1 = df1[df1['label'] == 1]

# Select a random sample from each label group
random_value_label_0 = df_label_0.sample(n=1).iloc[0]
random_value_label_1 = df_label_1.sample(n=1).iloc[0]

# Print the random values
print("Random value with label 0:")
print(random_value_label_0)

print("\nRandom value with label 1:")
print(random_value_label_1)

# These features from 2 random jobs are used below as examples.


Random value with label 0:
inst                     798643.0
bi                         3091.0
mem_req                       8.0
num_gpus_req                    1
num_cores_req                  16
num_tasks                    16.0
qos                             1
priority                   298700
eligible_time_epoch    1595485046
total_power                  2530
mean_power                    843
run_time                     53.0
label                         0.0
Name: 56580, dtype: object

Random value with label 1:
inst                   245516.790698
bi                        864.075323
mem_req                    237.75576
num_gpus_req                       4
num_cores_req                    128
num_tasks                        4.0
qos                                1
priority                      158434
eligible_time_epoch       1601983998
total_power                      860
mean_power                       860
run_time                    2.016206
label                           

The following code block contains a aggreagation of all models to exemplify the structure of the full model would have in the HPC cluster. This version is executable in the notebook.

In [13]:
import joblib
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import time  # Import the time module for measuring execution time
from sklearn.preprocessing import MinMaxScaler
from concurrent.futures import ThreadPoolExecutor

# Load all models and scalers once at the start
classifier_model = joblib.load('classifier_model.joblib')
scaler_classification = joblib.load('scaler_classification.joblib')

# Load regression scalers for runtime and mean power
scalers_regression = {}
for class_label in range(7):
    scaler_path = f'scaler_class_{class_label}.pkl'
    if os.path.exists(scaler_path):
        scalers_regression[class_label] = joblib.load(scaler_path)

# Load regression models for runtime
regressors_runtime = {class_label: joblib.load(f'regressor_model_class_{class_label}.pkl') for class_label in range(7)}

# Load mean power regression model and scaler
mean_power_model = joblib.load('regression_model.joblib')
mean_power_scaler = joblib.load('scaler_regression.joblib')
y_scaler = joblib.load('y_scaler.joblib')

# Load multiclass classifier and scaler once
stacking_clf = joblib.load('multiclass_stacking_classifier_model_6classes.pkl')
scaler_multiclass = joblib.load('minmax_scaler.pkl')

# Function to scale input for classification without 'num_tasks'
def scale_for_classification(input_data):
    features_to_scale = ['inst', 'bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'qos', 'priority', 'eligible_time_epoch']
    input_array = np.array([[input_data[feature] for feature in features_to_scale]])
    X_scaled = scaler_classification.transform(input_array)
    return X_scaled

# Prediction workflow function
def predict_workflow(inputs):
    try:
        # Step 1: Scale input data for binary classification (excluding 'num_tasks')
        X_scaled_classification = scale_for_classification(inputs)

        # Step 2: Binary classification
        classification = classifier_model.predict(X_scaled_classification)[0]

        # If classification predicts 1, return and stop the process
        if classification == 1:
            return {
                'classification': 1,
                'predicted_class': None,
                'predicted_runtime': None,
                'predicted_mean_power': None
            }

        # Step 3: Predict mean power
        input_array = np.array([[inputs['inst'], inputs['bi'], inputs['num_gpus_req'], inputs['mem_req'],
                                 inputs['num_cores_req'], inputs['num_tasks'], inputs['qos'],
                                 inputs['priority'], inputs['eligible_time_epoch']]])
        
        X_scaled_power = mean_power_scaler.transform(input_array)
        predicted_mean_power_scaled = mean_power_model.predict(X_scaled_power)
        predicted_mean_power = y_scaler.inverse_transform(predicted_mean_power_scaled.reshape(-1, 1))[0, 0]

        # Step 4: Multiclass classification
        X_scaled_multiclass = scaler_multiclass.transform(input_array)
        predicted_class = stacking_clf.predict(X_scaled_multiclass)[0]

        # Step 5: Select feature subset for regression
        feature_subsets = {
            0: ['num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch'],
            1: ['inst', 'num_gpus_req', 'num_cores_req', 'qos', 'eligible_time_epoch'],
            2: ['inst', 'bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'priority', 'eligible_time_epoch'],
            3: ['inst', 'bi', 'num_gpus_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch'],
            4: ['inst', 'bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch'],
            5: ['num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch'],
            6: ['bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch']
        }

        selected_features = feature_subsets[predicted_class]
        X_class = np.array([[inputs[feature] for feature in selected_features]])

        # Step 6: Scale features for regression
        scaler_runtime = scalers_regression.get(predicted_class, None)
        X_scaled_runtime = scaler_runtime.transform(X_class) if scaler_runtime else X_class

        # Step 7: Predict runtime
        regressor_runtime = regressors_runtime[predicted_class]
        predicted_runtime = regressor_runtime.predict(X_scaled_runtime)[0]

        return {
            'classification': 0,
            'predicted_class': predicted_class,
            'predicted_runtime': predicted_runtime,
            'predicted_mean_power': predicted_mean_power
        }

    except Exception as e:
        return {'error': str(e)}


# Example prediction for a single input
example_input_0 = {
    'inst': 798643,
    'bi': 3091,
    'num_gpus_req': 1,
    'mem_req': 8,
    'num_cores_req': 16,
    'num_tasks': 16,
    'qos': 1,
    'priority': 298700,
    'eligible_time_epoch': 1595485046
}


example_input_1 = {
    'inst': 245517,
    'bi': 864,
    'num_gpus_req': 4,
    'mem_req': 238,
    'num_cores_req': 128,
    'num_tasks': 4,
    'qos': 1,
    'priority': 158434,
    'eligible_time_epoch': 1601983998
}


# Measure time for prediction
start_time_0 = time.time()

# Single prediction for example inputs
result_0 = predict_workflow(example_input_0)

end_time_0 = time.time()
execution_time_0 = end_time_0 - start_time_0

# Print the prediction result and the time taken
print("Prediction result:", result_0)
print(f"Time taken for prediction: {execution_time_0:.4f} seconds")


# Measure time for prediction
start_time_1 = time.time()

# Single prediction for example inputs
result_1 = predict_workflow(example_input_1)

end_time_1 = time.time()
execution_time_1 = end_time_1 - start_time_1

# Print the prediction result and the time taken
print("Prediction result:", result_1)
print(f"Time taken for prediction: {execution_time_1:.4f} seconds")


Prediction result: {'classification': 0, 'predicted_class': 1, 'predicted_runtime': 49.56, 'predicted_mean_power': 847.5100000000002}
Time taken for prediction: 0.6258 seconds
Prediction result: {'classification': 1, 'predicted_class': None, 'predicted_runtime': None, 'predicted_mean_power': None}
Time taken for prediction: 0.0420 seconds


### NOTE: 
The code block below is a sketch of how the function would look like in an HPC cluster environment.

In [ ]:
import joblib
import numpy as np
import os
import time
import sys  # For command-line argument parsing
from sklearn.preprocessing import MinMaxScaler
import argparse  # For argument parsing

#_______________________________________________________________________________________________________________#
# Load all models and scalers once at the start (This would be done separately as it is necessary to do only once)
classifier_model = joblib.load('classifier_model.joblib')
scaler_classification = joblib.load('scaler_classification.joblib')

# Load regression scalers for runtime and mean power
scalers_regression = {}
for class_label in range(7):
    scaler_path = f'scaler_class_{class_label}.pkl'
    if os.path.exists(scaler_path):
        scalers_regression[class_label] = joblib.load(scaler_path)

# Load regression models for runtime
regressors_runtime = {class_label: joblib.load(f'regressor_model_class_{class_label}.pkl') for class_label in range(7)}

# Load mean power regression model and scaler
mean_power_model = joblib.load('regression_model.joblib')
mean_power_scaler = joblib.load('scaler_regression.joblib')
y_scaler = joblib.load('y_scaler.joblib')

# Load multiclass classifier and scaler once
stacking_clf = joblib.load('multiclass_stacking_classifier_model_6classes.pkl')
scaler_multiclass = joblib.load('minmax_scaler.pkl')

#_________________________________________________________________________________________________________________#


# Function to scale input for classification without 'num_tasks'
def scale_for_classification(input_data):
    features_to_scale = ['inst', 'bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'qos', 'priority', 'eligible_time_epoch']
    input_array = np.array([[input_data[feature] for feature in features_to_scale]])
    X_scaled = scaler_classification.transform(input_array)
    return X_scaled

# Prediction workflow function
def predict_workflow(inputs):
    try:
        # Scaling input data for binary classification (excluding 'num_tasks')
        X_scaled_classification = scale_for_classification(inputs)

        # Binary classification
        classification = classifier_model.predict(X_scaled_classification)[0]

        # If classification predicts 1, return and stop the process
        if classification == 1:
            return {
                'classification': 1,
                'predicted_class': None,
                'predicted_runtime': None,
                'predicted_mean_power': None
            }

        # Predict mean power
        input_array = np.array([[inputs['inst'], inputs['bi'], inputs['num_gpus_req'], inputs['mem_req'],
                                 inputs['num_cores_req'], inputs['num_tasks'], inputs['qos'],
                                 inputs['priority'], inputs['eligible_time_epoch']]])
        
        X_scaled_power = mean_power_scaler.transform(input_array)
        predicted_mean_power_scaled = mean_power_model.predict(X_scaled_power)
        predicted_mean_power = y_scaler.inverse_transform(predicted_mean_power_scaled.reshape(-1, 1))[0, 0]

        # Multiclass classification for run time prediction
        X_scaled_multiclass = scaler_multiclass.transform(input_array)
        predicted_class = stacking_clf.predict(X_scaled_multiclass)[0]

        # Select feature subset for regression in run time prediction
        feature_subsets = {
            0: ['num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch'],
            1: ['inst', 'num_gpus_req', 'num_cores_req', 'qos', 'eligible_time_epoch'],
            2: ['inst', 'bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'priority', 'eligible_time_epoch'],
            3: ['inst', 'bi', 'num_gpus_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch'],
            4: ['inst', 'bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch'],
            5: ['num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch'],
            6: ['bi', 'num_gpus_req', 'mem_req', 'num_cores_req', 'num_tasks', 'qos', 'priority', 'eligible_time_epoch']
        }

        selected_features = feature_subsets[predicted_class]
        X_class = np.array([[inputs[feature] for feature in selected_features]])

        # Scale features
        scaler_runtime = scalers_regression.get(predicted_class, None)
        X_scaled_runtime = scaler_runtime.transform(X_class) if scaler_runtime else X_class

        # Predict runtime
        regressor_runtime = regressors_runtime[predicted_class]
        predicted_runtime = regressor_runtime.predict(X_scaled_runtime)[0]

        return {
            'classification': 0,
            'predicted_class': predicted_class,
            'predicted_runtime': predicted_runtime,
            'predicted_mean_power': predicted_mean_power
        }

    except Exception as e:
        return {'error': str(e)}

# Command-line argument parsing function
def parse_args():
    parser = argparse.ArgumentParser(description="Run prediction before scheduling a job in the HPC cluster.")
    parser.add_argument('--inst', type=int, required=True, help="Instruction count")
    parser.add_argument('--bi', type=int, required=True, help="Branch instruction count")
    parser.add_argument('--num_gpus_req', type=int, required=True, help="Number of GPUs required")
    parser.add_argument('--mem_req', type=int, required=True, help="Memory required (in GB)")
    parser.add_argument('--num_cores_req', type=int, required=True, help="Number of cores required")
    parser.add_argument('--num_tasks', type=int, required=True, help="Number of tasks")
    parser.add_argument('--qos', type=int, required=True, help="QoS level")
    parser.add_argument('--priority', type=int, required=True, help="Job priority")
    parser.add_argument('--eligible_time_epoch', type=int, required=True, help="Eligible time in epoch format")
    
    return parser.parse_args()

# Main function to run prediction
def main():
    args = parse_args()

    input_data = {
        'inst': args.inst,
        'bi': args.bi,
        'num_gpus_req': args.num_gpus_req,
        'mem_req': args.mem_req,
        'num_cores_req': args.num_cores_req,
        'num_tasks': args.num_tasks,
        'qos': args.qos,
        'priority': args.priority,
        'eligible_time_epoch': args.eligible_time_epoch
    }

    # Measure time for prediction
    start_time = time.time()

    # Run prediction
    result = predict_workflow(input_data)

    end_time = time.time()
    execution_time = end_time - start_time

    # Output the prediction result and the time taken
    print("Prediction result:", result)
    print(f"Time taken for prediction: {execution_time:.4f} seconds")

if __name__ == "__main__":
    main()
